# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc
import json

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence

from torch import cuda

from models.BERT_Dropout_CNN_BiLSTM_Linear import BERT_Dropout_CNN_BiLSTM_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
DATASET = 'ATE_SemEval16_Restaurants_train.json'

In [7]:
df = pd.json_normalize(json.load(open(DATASET)))

In [8]:
df.head()

,text,tokens,iob_aspect_tags
0,Judging from previous posts this used to be a ...,"[Judging, from, previous, posts, this, used, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
1,"We, there were four of us, arrived at noon - t...","[We, ,, there, were, four, of, us, ,, arrived,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"They never brought us complimentary noodles, i...","[They, never, brought, us, complimentary, nood...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,The food was lousy - too sweet or too salty an...,"[The, food, was, lousy, -, too, sweet, or, too...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"


# Train & Validate

In [9]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [10]:
SEQ_LEN = 512

In [11]:
BERT_FINE_TUNED_PATH = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [12]:
MODEL_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned_dropout_cnn_bilstm_linear.pth'
STATS_OUTPUT = '../../../results/ATE/SemEval16 - Task 5 - Restaurants/stats/bert_fine_tuned_dropout_cnn_bilstm_linear.csv'

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
best_accuracy = 0.0

In [15]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(ids_tensors[0])), 0)(ids_tensors[0])
    ids_tensors = pad_sequence(ids_tensors, batch_first=True).to(device)

    tags_tensors = [s[2] for s in samples]
    tags_tensors[0] = torch.nn.ConstantPad1d((0, SEQ_LEN - len(tags_tensors[0])), 0)(tags_tensors[0])
    tags_tensors = pad_sequence(tags_tensors, batch_first=True).to(device)
    
    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long).to(device)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1).to(device)
    
    return ids_tensors, tags_tensors, masks_tensors

In [16]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids_tensors, tags_tensors, masks_tensors = data

        outputs = model(ids_tensors, masks_tensors)

        loss = loss_fn(outputs.view(-1, 3), tags_tensors.view(-1))
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [17]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids_tensors, tags_tensors, masks_tensors = data
            ids_tensors = ids_tensors.to(device)
            tags_tensors = tags_tensors.to(device)
            masks_tensors = masks_tensors.to(device)

            outputs = model(ids_tensors, masks_tensors)
            
            _, predictions = torch.max(outputs, dim=2)

            fin_outputs += list([int(p) for pred in predictions for p in pred])
            fin_targets += list([int(tag) for tags_tensor in tags_tensors for tag in tags_tensor])

    return fin_outputs, fin_targets

In [18]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [19]:
for i in range(NO_RUNS):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/{NO_RUNS}")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True,
        collate_fn=create_mini_batch
    )

    model = BERT_Dropout_CNN_BiLSTM_Linear(torch.load(BERT_FINE_TUNED_PATH), bert_seq_len=SEQ_LEN, dropout=0.3, bilstm_in_features=256, conv_out_channels=SEQ_LEN, conv_kernel_size=3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1272287368774414
Epoch: 0/2, Batch: 50/501, Loss: 0.504747211933136
Epoch: 0/2, Batch: 100/501, Loss: 0.1400931477546692
Epoch: 0/2, Batch: 150/501, Loss: 0.06927359849214554
Epoch: 0/2, Batch: 200/501, Loss: 0.038700785487890244
Epoch: 0/2, Batch: 250/501, Loss: 0.048374325037002563
Epoch: 0/2, Batch: 300/501, Loss: 0.0359799824655056
Epoch: 0/2, Batch: 350/501, Loss: 0.028781231492757797
Epoch: 0/2, Batch: 400/501, Loss: 0.027132274582982063
Epoch: 0/2, Batch: 450/501, Loss: 0.021517794579267502
Epoch: 0/2, Batch: 500/501, Loss: 0.022560089826583862
Epoch: 1/2, Batch: 0/501, Loss: 0.04680657759308815
Epoch: 1/2, Batch: 50/501, Loss: 0.02364029921591282
Epoch: 1/2, Batch: 100/501, Loss: 0.018369892612099648
Epoch: 1/2, Batch: 150/501, Loss: 0.017621204257011414
Epoch: 1/2, Batch: 200/501, Loss: 0.018922442570328712
Epoch: 1/2, Batch: 250/501, Loss: 0.023588869720697403
Epoch: 1/2, Batch: 300/501, Loss: 0.018757443875074387
Epoch: 1/2, Batch: 

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 8/10
Epoch: 0/2, Batch: 0/501, Loss: 1.1305608749389648
Epoch: 0/2, Batch: 50/501, Loss: 0.6343294978141785
Epoch: 0/2, Batch: 100/501, Loss: 0.1676105111837387
Epoch: 0/2, Batch: 150/501, Loss: 0.07165433466434479
Epoch: 0/2, Batch: 200/501, Loss: 0.04187655821442604
Epoch: 0/2, Batch: 250/501, Loss: 0.033249273896217346
Epoch: 0/2, Batch: 300/501, Loss: 0.027294183149933815
Epoch: 0/2, Batch: 350/501, Loss: 0.02317405492067337
Epoch: 0/2, Batch: 400/501, Loss: 0.0308480653911829
Epoch: 0/2, Batch: 450/501, Loss: 0.03605445474386215
Epoch: 0/2, Batch: 500/501, Loss: 0.025922199711203575
Epoch: 1/2, Batch: 0/501, Loss: 0.026304323226213455
Epoch: 1/2, Batch: 50/501, Loss: 0.025276776403188705
Epoch: 1/2, Batch: 100/501, Loss: 0.01078348234295845
Epoch: 1/2, Batch: 150/501, Loss: 0.01691964641213417
Epoch: 1/2, Batch: 200/501, Loss: 0.018830932676792145
Epoch: 1/2, Batch: 250/501, Loss: 0.017202500253915787
Epoch: 1/2, Batch: 300/501, Loss: 0.01230141893029213
Epoch: 1/2, Batch: 350

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/501, Loss: 1.137380599975586
Epoch: 0/2, Batch: 50/501, Loss: 0.7412877678871155
Epoch: 0/2, Batch: 100/501, Loss: 0.2812727689743042
Epoch: 0/2, Batch: 150/501, Loss: 0.1299826055765152
Epoch: 0/2, Batch: 200/501, Loss: 0.09861220419406891
Epoch: 0/2, Batch: 250/501, Loss: 0.07226618379354477
Epoch: 0/2, Batch: 300/501, Loss: 0.061550747603178024
Epoch: 0/2, Batch: 350/501, Loss: 0.056143663823604584
Epoch: 0/2, Batch: 400/501, Loss: 0.05792200565338135
Epoch: 0/2, Batch: 450/501, Loss: 0.04020576551556587
Epoch: 0/2, Batch: 500/501, Loss: 0.0824773907661438
Epoch: 1/2, Batch: 0/501, Loss: 0.03838009759783745
Epoch: 1/2, Batch: 50/501, Loss: 0.03935406729578972
Epoch: 1/2, Batch: 100/501, Loss: 0.036269329488277435
Epoch: 1/2, Batch: 150/501, Loss: 0.051103074103593826
Epoch: 1/2, Batch: 200/501, Loss: 0.04897179827094078
Epoch: 1/2, Batch: 250/501, Loss: 0.03364236652851105
Epoch: 1/2, Batch: 300/501, Loss: 0.030694471672177315
Epoch: 1/2, Batch: 350/50

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.995188,0.995188,0.631969,0.995188,0.353304,0.995188,0.369719,363.689629
1,0.995359,0.995359,0.619140,0.995359,0.372968,0.995359,0.402289,360.759269
2,0.994969,0.994969,0.643920,0.994969,0.366024,0.994969,0.390377,360.192474
3,0.995227,0.995227,0.532637,0.995227,0.364972,0.995227,0.384270,368.186726
4,0.995559,0.995559,0.713497,0.995559,0.398690,0.995559,0.429373,377.635806
5,0.995316,0.995316,0.627876,0.995316,0.373912,0.995316,0.401551,384.244402
6,0.994934,0.994934,0.331647,0.994934,0.333332,0.994934,0.332487,361.655894
7,0.994707,0.994707,0.551313,0.994707,0.384984,0.994707,0.409239,361.001662
8,0.994957,0.994957,0.498425,0.994957,0.362664,0.994957,0.382483,360.938213
9,0.994984,0.994984,0.331661,0.994984,0.333333,0.994984,0.332495,360.835322


In [21]:
results.to_csv(STATS_OUTPUT)